In [286]:
import gzip
import json
import nltk
from nltk.corpus import stopwords 
nltk.download()

#Reference: BOW and NLP credits to https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words
data_size = 10000

def review_to_words(review_text):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    #review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

with gzip.open('9.28_9.30_labor.jsonl.gz', 'rt') as f:
    ad = 0
    clean_train_reviews = []
    for line in f:
        
         # remove line break
        line_cleaned = line.split('\n')[0]
        
        # read in line as JSON object
        # note that since these documents were stored in Elasticsearch 
        # you need to access the `_source` to get the actual object
        # source contains the actual CDR schema object
                
        obj = json.loads(line_cleaned).get('_source')
        
        # you can obtain the `_id` in case it is useful
        # it is stored at the same level as `_source`
        
        _extractwords = json.loads(line_cleaned).get('_source').get('extracted_text')
        
        clean_train_reviews.append(review_to_words( _extractwords ))
        ad = ad + 1 
        if ad == data_size:
            break
        # now you can operate on `obj` as needed 
        
with gzip.open('september_nyc_escorts.jsonl.gz', 'rt') as f:
    ad = 0
    clean_train_escort_reviews = []
    for line in f:
        
         # remove line break
        line_cleaned = line.split('\n')[0]
        
        # read in line as JSON object
        # note that since these documents were stored in Elasticsearch 
        # you need to access the `_source` to get the actual object
        # source contains the actual CDR schema object
                
        obj = json.loads(line_cleaned).get('_source')
        
        # you can obtain the `_id` in case it is useful
        # it is stored at the same level as `_source`
        
        _extractwords = json.loads(line_cleaned).get('_source').get('extracted_text')
        
        clean_train_escort_reviews.append(review_to_words( _extractwords ))
        ad = ad + 1 
        if ad == data_size:
            break
        # now you can operate on `obj` as needed 

In [287]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 500) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_size = int(4/5 * data_size)
test_size = data_size - train_size
train_data_all = np.concatenate((clean_train_reviews[:train_size],clean_train_escort_reviews[:train_size]), axis=0)
train_data_features = vectorizer.fit_transform(train_data_all)
#vectorizer.transform()
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
train_data_features.shape



import numpy as np 
length = train_size
target_variable = np.ones(length)
target_variable_1 = -1 * np.ones(length)
tv = np.concatenate((target_variable, target_variable_1),axis=0)
tv.shape

Creating the bag of words...



(16000,)

In [288]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(train_data_features,tv)

Training the random forest...


In [289]:

test_data_features = vectorizer.transform(clean_train_escort_reviews[train_size:])
result = forest.predict(test_data_features)

In [290]:
np.mean(result == -1 )

0.999

In [291]:
text1 = 'Names Tiffany 19 y.o. female from Queens NY .. Looking For A G-e-ne-r-o-.u.s man who wants to have a little quick fun .. Email me for more details'
string_text = [review_to_words(text1)]
test_data = vectorizer.transform(string_text)
string_text2 = [review_to_words(text2)]
tesr_data_features = test_data.toarray()
result = forest.predict(test_data)
result

array([-1.])